In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from Alcohol.caching import load
from Processing import TimeSeries, PriceQuantity as PQ


In [ ]:
ts = load(dir_path="../Alcohol/Caches", identifier="0")
longest_ts = ts.get_nth_longest(0)
top_n = [ts.get_nth_longest(i) for i in range(10)]
ts.price.shape

In [ ]:
for element in top_n:
    print("normal: ", np.corrcoef(x=element.price, y=element.quantity)[0, 1])
    print("demeaned: ", np.corrcoef(x=element.price, y=PQ.skip_demean_quantity(element, period=12).quantity)[0, 1])
    shifted = PQ.shift_price(element, 1)
    print("shifted: ", np.corrcoef(x=shifted.price, y=PQ.skip_demean_quantity(shifted, period=12).quantity)[0, 1])
    print("---------")


In [ ]:
for element in top_n:
    demeaned = PQ.skip_demean_quantity(element, period=12)
    demeaned.draw_scatter_graph(plt.subplots()[1], economic_axis=True)
plt.show()

In [ ]:
   
demean = True
shift = False
for i, pq in enumerate(top_n):
    if demean:
        pq = PQ.skip_demean_quantity(pq, 12)
    if shift:
        pq = PQ.shift_price(pq, -1)
    print(f'{i} ', pq.get_correlation())
    pq.draw_time_axis_scatter_graph(plt.subplots()[1], dates=True)
    plt.show()  
5
        

In [ ]:
from Processing import ModelPQ

pq = top_n[7]
pq = PQ.bin_price_absolute(pq, 0.25)[:]
print(pq.price)
#print(pq.quantity)
#print(find_change_range(pq))
# (
#     last_change_slope(pq, min_count=8),
#     last_change_with_time_slope(pq, min_count=15),
#     linear_model_slope(pq),
#     linear_model_with_time_slope(pq)
#  )

model = ModelPQ(pq, min_count=10)
model.run_models()
model.results
